In [3]:
# !pip -q install openpyxl

from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

def create_invoice_xlsx():
    # --- 日付・ファイル名 ---
    today = datetime.now()
    issue_date_str = today.strftime("%Y/%m/%d")   # 表示用（YYYY/MM/DD）
    file_date_str  = today.strftime("%Y%m%d")     # ファイル名用（YYYYMMDD）
    filename = f"請求書_{file_date_str}.xlsx"

    # --- ワークブック作成 ---
    wb = Workbook()
    ws = wb.active
    ws.title = "請求書"

    # --- 体裁（列幅） ---
    col_widths = {
        "A": 5,   # No
        "B": 26,  # 品名
        "C": 10,  # 数量
        "D": 12,  # 単価
        "E": 12,  # 金額
        "F": 18,  # 備考
    }
    for col, w in col_widths.items():
        ws.column_dimensions[col].width = w

    # --- スタイル定義 ---
    title_font = Font(size=20, bold=True)
    header_font = Font(bold=True, color="FFFFFF")
    bold_font = Font(bold=True)

    center = Alignment(horizontal="center", vertical="center")
    right  = Alignment(horizontal="right",  vertical="center")
    left   = Alignment(horizontal="left",   vertical="center")

    fill_header = PatternFill("solid", fgColor="4F81BD")  # 濃い青
    fill_total  = PatternFill("solid", fgColor="D9E1F2")  # 薄い青

    thin = Side(style="thin", color="000000")
    border_thin = Border(left=thin, right=thin, top=thin, bottom=thin)

    # --- タイトル ---
    ws.merge_cells("A1:F2")
    ws["A1"] = "請求書"
    ws["A1"].font = title_font
    ws["A1"].alignment = center

    # --- 発行者（固定データ） ---
    ws["A3"] = "株式会社ABC"
    ws["A3"].font = bold_font
    ws["A4"] = "〒101-0022 東京都千代田区神田練塀町300"
    ws["A5"] = "TEL:03-1234-5678 FAX:03-1234-5678"

    # --- 宛先（固定データ） ---
    ws["A7"] = "担当者名:鈴木一郎 様"
    ws["A7"].font = bold_font

    # --- 日付（現在日付・YYYY/MM/DD） ---
    ws.merge_cells("E3:F3")
    ws["E3"] = f"日付：{issue_date_str}"
    ws["E3"].alignment = right

    # --- 請求明細（例：必要に応じて編集） ---
    # 品名, 数量, 単価, 備考
    items = [
        ("商品A", 2, 15000, ""),
        ("商品B", 1,  8000, ""),
        ("作業費", 3,  5000, "6月分"),
    ]

    start_row = 11
    headers = ["No", "品名", "数量", "単価", "金額", "備考"]

    # ヘッダ行
    for j, h in enumerate(headers, start=1):
        cell = ws.cell(row=start_row, column=j, value=h)
        cell.font = header_font
        cell.fill = fill_header
        cell.alignment = center
        cell.border = border_thin

    # 明細行
    subtotal = 0
    for i, (name, qty, unit_price, note) in enumerate(items, start=1):
        r = start_row + i
        amount = qty * unit_price
        subtotal += amount

        row_values = [i, name, qty, unit_price, amount, note]
        for c, v in enumerate(row_values, start=1):
            cell = ws.cell(row=r, column=c, value=v)
            cell.border = border_thin
            cell.alignment = center if c in (1,3) else (right if c in (4,5) else left)

            # 数値書式
            if c in (4,5):  # 単価・金額
                cell.number_format = "#,##0"

    # --- 小計・消費税・合計 ---
    tax_rate = 0.10
    tax = int(round(subtotal * tax_rate))
    total = subtotal + tax

    sum_start = start_row + len(items) + 2
    labels = [("小計", subtotal), ("消費税(10%)", tax), ("合計", total)]

    for k, (label, value) in enumerate(labels):
        r = sum_start + k
        ws.merge_cells(f"A{r}:D{r}")
        ws[f"A{r}"] = label
        ws[f"A{r}"].alignment = right
        ws[f"A{r}"].font = bold_font

        ws.merge_cells(f"E{r}:F{r}")
        ws[f"E{r}"] = value
        ws[f"E{r}"].alignment = right
        ws[f"E{r}"].number_format = "#,##0"
        ws[f"E{r}"].font = bold_font

        # 罫線
        for col in range(1, 7):
            ws.cell(row=r, column=col).border = border_thin

        # 合計行だけ薄い塗り
        if label == "合計":
            for col in range(1, 7):
                ws.cell(row=r, column=col).fill = fill_total

    # --- 印刷向け微調整（任意） ---
    ws.page_setup.orientation = ws.ORIENTATION_PORTRAIT
    ws.page_setup.fitToWidth = 1
    ws.page_setup.fitToHeight = 1

    # --- 保存 ---
    wb.save(filename)
    return filename

# 実行
fname = create_invoice_xlsx()
print("作成しました:", fname)

作成しました: 請求書_20260102.xlsx
